In [1]:
# Import packages.
import msprime, pyslim, numpy

In [2]:
# Create a very basic msprime setup with population structure, test.

population_configurations =[
    msprime.PopulationConfiguration(
    sample_size=3, initial_size=100),
    msprime.PopulationConfiguration(
    sample_size=3, initial_size=100)
]

demographic_events = [
    msprime.MassMigration(
    time=5, source=1, destination=0, proportion=1.0)
]

rho = 1e-4
mu  = 1e-3 

dd = msprime.DemographyDebugger(
    population_configurations=population_configurations,
    demographic_events=demographic_events
)
dd.print_history()

Epoch: 0 -- 5.0 generations
     start     end      growth_rate |     0        1    
   -------- --------       -------- | -------- -------- 
0 |   100      100                0 |     0        0    
1 |   100      100                0 |     0        0    

Events @ generation 5.0
   - Mass migration: lineages move from 1 to 0 with probability 1.0
Epoch: 5.0 -- inf generations
     start     end      growth_rate |     0        1    
   -------- --------       -------- | -------- -------- 
0 |   100      100                0 |     0        0    
1 |   100      100                0 |     0        0    



In [3]:
# Simulate with msprime.

ts = msprime.simulate(
    population_configurations=population_configurations,
    demographic_events=demographic_events,
    length=10,
    recombination_rate=rho,
    mutation_rate=mu,
    random_seed=1
)

In [4]:
for tree in ts.trees():
    print(tree.draw(format="unicode"))

   11        
┏━━┻━━┓      
┃     10     
┃  ┏━━┻━┓    
┃  ┃    9    
┃  ┃  ┏━┻┓   
┃  8  ┃  ┃   
┃ ┏┻┓ ┃  ┃   
┃ ┃ ┃ ┃  6   
┃ ┃ ┃ ┃ ┏┻┓  
2 0 1 3 4 5  

   11        
┏━━┻━━┓      
┃     10     
┃   ┏━┻━━┓   
┃   ┃    9   
┃   ┃   ┏┻┓  
┃   8   ┃ ┃  
┃ ┏━┻┓  ┃ ┃  
┃ ┃  7  ┃ ┃  
┃ ┃ ┏┻┓ ┃ ┃  
2 0 1 5 3 4  



Let's now investigate the `TreeSequence` class a little further, following the tutorial in the [msprime documentation](https://msprime.readthedocs.io/en/latest/api.html#using-simulation-results).

TreeSequence objects can be generated in `msprime` (or `SLiM`), and loaded from existing `.trees` files,`.txt` files or `TableCollection` objects. See the text for more.

Note that TreeSequences are immutable; you can only change the data held in a particular TreeSequence indirectly, by getting the table information, editing the tables and then creating a TreeSequence with the new set of tables using `TableCollection.tree_sequence()`. Perhaps let's investigate that later.

First, let's look at all of the sub-classes and constants contained within a TreeSequence:

In [5]:
dir(ts)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_ll_tree_sequence',
 'breakpoints',
 'diffs',
 'dump',
 'dump_tables',
 'dump_text',
 'edge_diffs',
 'edges',
 'edgesets',
 'file_uuid',
 'first',
 'genotype_matrix',
 'get_ll_tree_sequence',
 'get_num_mutations',
 'get_num_nodes',
 'get_num_records',
 'get_num_sites',
 'get_num_trees',
 'get_pairwise_diversity',
 'get_population',
 'get_sample_size',
 'get_samples',
 'get_sequence_length',
 'get_time',
 'haplotypes',
 'individual',
 'individuals',
 'll_tree_sequence',
 'load',
 'load_tables',
 'migrations',
 'mutation',
 'mutations',
 'newick_trees',
 'node',
 'nodes',
 'num_edges',
 'num_individuals',
 'num_migrations'

Firstly, some useful summary info about the TreeSequence:

In [31]:
print("number of edges: ", ts.num_edges, "\n")
print("number of individuals: ", ts.num_individuals, "\n")
print("number of migrations: ", ts.num_migrations, "\n")
print("number of mutations: ", ts.num_mutations, "\n")
print("number of nodes: ", ts.num_nodes, "\n")
print("number of populations: ", ts.num_populations, "\n")
print("number of provenances: ", ts.num_provenances, "\n")
print("number of samples: ", ts.num_samples, "\n")
print("number of sites: ", ts.num_sites, "\n")
print("number of trees: ", ts.num_trees, "\n")
print("sequence length: ", ts.sequence_length, "\n")

number of edges:  14 

number of individuals:  0 

number of migrations:  0 

number of mutations:  14 

number of nodes:  12 

number of populations:  2 

number of provenances:  1 

number of samples:  6 

number of sites:  14 

number of trees:  2 

sequence length:  10.0 



**NOTE!** The sequence length is NOT the number of segregating sites - it is with respect to whatever unit you specify as input into ```msprime.simulate()```.

First, let's examine the TreeSequence. 

The topology of the tree indicates the relationships of each node to each other node. These can be drawn using `draw()` (although for complicated trees, this sort of screws up - see below).

Printing the tree directly outputs a list showing all child:parent node relationships.

In [7]:
for tree in ts.trees():
    print(tree)

{0: 8, 1: 8, 2: 11, 3: 9, 4: 6, 5: 6, 6: 9, 8: 10, 9: 10, 10: 11}
{0: 8, 1: 7, 2: 11, 3: 9, 4: 9, 5: 7, 7: 8, 8: 10, 9: 10, 10: 11}


In [8]:
for tree in ts.trees():
    print(tree.draw(format="unicode"))

   11        
┏━━┻━━┓      
┃     10     
┃  ┏━━┻━┓    
┃  ┃    9    
┃  ┃  ┏━┻┓   
┃  8  ┃  ┃   
┃ ┏┻┓ ┃  ┃   
┃ ┃ ┃ ┃  6   
┃ ┃ ┃ ┃ ┏┻┓  
2 0 1 3 4 5  

   11        
┏━━┻━━┓      
┃     10     
┃   ┏━┻━━┓   
┃   ┃    9   
┃   ┃   ┏┻┓  
┃   8   ┃ ┃  
┃ ┏━┻┓  ┃ ┃  
┃ ┃  7  ┃ ┃  
┃ ┃ ┏┻┓ ┃ ┃  
2 0 1 5 3 4  



Because the TreeSequence is stored compactly as an iterator object, you can't simply access the trees by subsetting the object. Each subsequent tree is built upon the information in the current tree, so they cannot be accessed out of order.

However, it is sometimes useful to visualise a smaller part of the TreeSequence for basic sanity checks. For this purpose you can use `first()` to print just the first tree in the TreeSequence.

In [9]:
print(ts.first())

{0: 8, 1: 8, 2: 11, 3: 9, 4: 6, 5: 6, 6: 9, 8: 10, 9: 10, 10: 11}


In [10]:
print(ts.first().draw(format="unicode"))

   11        
┏━━┻━━┓      
┃     10     
┃  ┏━━┻━┓    
┃  ┃    9    
┃  ┃  ┏━┻┓   
┃  8  ┃  ┃   
┃ ┏┻┓ ┃  ┃   
┃ ┃ ┃ ┃  6   
┃ ┃ ┃ ┃ ┏┻┓  
2 0 1 3 4 5  



A single tree in a TreeSequence is a SparseTree. This is an object of it's own type, to be considered in a separate notebook!

Each of these trees depicts the genealogical history of a particular genomic interval along the simulated chromosome. To see the intervals, use  `breakpoints()`. The intervals spanning each consecutive pair of coordinates in this list each have the same history with respect to the sample, corresponding to each tree in the above TreeSequence.

Note that the breakpoints have a floating coordinate. This is because, at present msprime model throws down recombination events (and mutation events) at continuous points along the simulated chromosome, even if you specify an integer chromosome length in `msprime.simulate()` (as wel have done above). This is important to keep in mind for later, when we attempt to initialise `msprime` with a `SLiM`-generated .trees file.

In [11]:
for pt in ts.breakpoints():
    print(pt)

0
8.805353552756214
10.0


Each node in these trees corresponds to the genome of a specific ancestor at some time in the past. (See the [msprime documentation](https://msprime.readthedocs.io/en/latest/interchange.html#node-table) for more description). These can be accessed sequentially, or individually: 

In [12]:
for node in ts.nodes():
    print(node)

{'id': 0, 'time': 0.0, 'population': 0, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 1, 'time': 0.0, 'population': 0, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 2, 'time': 0.0, 'population': 0, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 3, 'time': 0.0, 'population': 1, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 4, 'time': 0.0, 'population': 1, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 5, 'time': 0.0, 'population': 1, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 6, 'time': 65.92482819029316, 'population': 0, 'individual': -1, 'metadata': b'', 'flags': 0}
{'id': 7, 'time': 71.57784790772325, 'population': 0, 'individual': -1, 'metadata': b'', 'flags': 0}
{'id': 8, 'time': 87.05704331323707, 'population': 0, 'individual': -1, 'metadata': b'', 'flags': 0}
{'id': 9, 'time': 125.58636700016072, 'population': 0, 'individual': -1, 'metadata': b'', 'flags': 0}
{'id': 10, 'time': 265.89952008528115, 'population': 0, 'individual': -1,

In [13]:
ts.node(6)

{'id': 6, 'time': 65.92482819029316, 'population': 0, 'individual': -1, 'metadata': b'', 'flags': 0}

We're often most interested in the nodes that correspond to the present-day samples whose genomes we've simulated. To see the ids corresponding to these, use ```samples()```:

In [28]:
ts.samples()

array([0, 1, 2, 3, 4, 5], dtype=int32)

Note that the two trees have a very similar topology.

Call `edges()` to see each unique edge (parent:child relationship) of the TreeSequence. Each row of this output corresponds to a single edge. The parent and child nodes of the edge are listed, as well as the endpoints of the genomic interval over which the edge ranges.

Note that shared aspects of the topologies of consecutive trees need only be recorded once. For instance, both of these trees share an edge between parent node 11 and child node 2. This is listed in a single row; the fact that this edge is a feature of the entire sequence's history is recorded by the endpoints of the interval (0,10). 

This is the main reason why TreeSequences are so efficient! 

In [14]:
for edge in ts.edges():
    print(edge)

{left=0.000, right=8.805, parent=6, child=4}
{left=0.000, right=8.805, parent=6, child=5}
{left=8.805, right=10.000, parent=7, child=1}
{left=8.805, right=10.000, parent=7, child=5}
{left=0.000, right=10.000, parent=8, child=0}
{left=0.000, right=8.805, parent=8, child=1}
{left=8.805, right=10.000, parent=8, child=7}
{left=0.000, right=10.000, parent=9, child=3}
{left=8.805, right=10.000, parent=9, child=4}
{left=0.000, right=8.805, parent=9, child=6}
{left=0.000, right=10.000, parent=10, child=8}
{left=0.000, right=10.000, parent=10, child=9}
{left=0.000, right=10.000, parent=11, child=2}
{left=0.000, right=10.000, parent=11, child=10}


To see how to construct each tree from the previous tree, moving left-to-right, use `edge_diffs()`:

In [15]:
for edge in ts.edge_diffs():
    print(edge, "\n")

((0.0, 8.805353552756214), [], [{left=0.000, right=8.805, parent=6, child=4}, {left=0.000, right=8.805, parent=6, child=5}, {left=0.000, right=10.000, parent=8, child=0}, {left=0.000, right=8.805, parent=8, child=1}, {left=0.000, right=10.000, parent=9, child=3}, {left=0.000, right=8.805, parent=9, child=6}, {left=0.000, right=10.000, parent=10, child=8}, {left=0.000, right=10.000, parent=10, child=9}, {left=0.000, right=10.000, parent=11, child=2}, {left=0.000, right=10.000, parent=11, child=10}]) 

((8.805353552756214, 10.0), [{left=0.000, right=8.805, parent=9, child=6}, {left=0.000, right=8.805, parent=8, child=1}, {left=0.000, right=8.805, parent=6, child=5}, {left=0.000, right=8.805, parent=6, child=4}], [{left=8.805, right=10.000, parent=7, child=1}, {left=8.805, right=10.000, parent=7, child=5}, {left=8.805, right=10.000, parent=8, child=7}, {left=8.805, right=10.000, parent=9, child=4}]) 



At the first tree spanning `(0.0, 8.8)`, there is no previous tree to modify, so no edges need to be removed to construct this tree. This is why the second value of the first edge difference is `[]`, the empty tree. 

The final element in this tuple is the entire list of edges needed to construct the tree. Since this is the first tree in the sequence, every edge must be provided.

Next, let's examine the second tuple corresponding to the second tree in this TreeSequence. This tree spans the interval `(8.8, 10.0)`, which is the first element in the tuple.

The second element is a list of edges indicating the parts of the previous tree that must be removed to construct the current tree. As my hand-drawn diagram indicates, the edges that must be removed are those connecting the nodes 9:6, 8:1,  6:5, 6:4. 

The third element is a list of edges indicating the parts of the current tree that must be added to create it. These are the edges connecting nodes 7:1, 7:5, 8:7, 9:4. 

**How to access the entire sample of simulated genotypes?** 

Simply use `genotype_matrix()`. Columns are individual haplotypes, rows are the variants. Note that only segregating sites are printed.

In [16]:
ts.genotype_matrix()

array([[0, 0, 1, 0, 0, 0],
       [1, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 1, 0, 1, 1, 1],
       [0, 0, 1, 0, 0, 0],
       [1, 1, 0, 1, 1, 1],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0]], dtype=uint8)

At each of these sites, a mutation has occured somewhere in the tree, leading to differences in the genotypes inherited by different leaf nodes.  Information about these mutations can be accessed sequentially via the `mutations()` iterator...

In [17]:
for mut in ts.mutations():
    print(mut)

{'id': 0, 'site': 0, 'node': 2, 'derived_state': '1', 'parent': -1, 'metadata': b'', 'position': 0.18288280116394162, 'index': 0}
{'id': 1, 'site': 1, 'node': 8, 'derived_state': '1', 'parent': -1, 'metadata': b'', 'position': 0.592431970871985, 'index': 1}
{'id': 2, 'site': 2, 'node': 1, 'derived_state': '1', 'parent': -1, 'metadata': b'', 'position': 0.8171439440980768, 'index': 2}
{'id': 3, 'site': 3, 'node': 10, 'derived_state': '1', 'parent': -1, 'metadata': b'', 'position': 2.2212454839609563, 'index': 3}
{'id': 4, 'site': 4, 'node': 2, 'derived_state': '1', 'parent': -1, 'metadata': b'', 'position': 3.1551563390530646, 'index': 4}
{'id': 5, 'site': 5, 'node': 10, 'derived_state': '1', 'parent': -1, 'metadata': b'', 'position': 4.461345006711781, 'index': 5}
{'id': 6, 'site': 6, 'node': 1, 'derived_state': '1', 'parent': -1, 'metadata': b'', 'position': 4.562516381453547, 'index': 6}
{'id': 7, 'site': 7, 'node': 9, 'derived_state': '1', 'parent': -1, 'metadata': b'', 'position': 

... or individually via the `mutation()` command:

** NOTE! It is a bit weird that some of the information is truncated (including the position info???)**

In [18]:
print(ts.mutation(10))

{'id': 10, 'site': 10, 'node': 1, 'derived_state': '1', 'parent': -1, 'metadata': b''}


In this toy example, the simulated dataset is quite small, so it's no big deal to access them all at once. However, if they aren't all needed at once, you can also access them sequentially using the `variants()` iterator. This can save a lot of memory. Note that other information is also recorded in `variants()`, including position, alleles, site indices and so on.

In [19]:
for var in ts.variants():
    print(var)

{'site': {'id': 0, 'position': 0.18288280116394162, 'ancestral_state': '0', 'mutations': [{'id': 0, 'site': 0, 'node': 2, 'derived_state': '1', 'parent': -1, 'metadata': b''}], 'metadata': b''}, 'alleles': ('0', '1'), 'genotypes': array([0, 0, 1, 0, 0, 0], dtype=uint8), 'position': 0.18288280116394162, 'index': 0}
{'site': {'id': 1, 'position': 0.592431970871985, 'ancestral_state': '0', 'mutations': [{'id': 1, 'site': 1, 'node': 8, 'derived_state': '1', 'parent': -1, 'metadata': b''}], 'metadata': b''}, 'alleles': ('0', '1'), 'genotypes': array([1, 1, 0, 0, 0, 0], dtype=uint8), 'position': 0.592431970871985, 'index': 1}
{'site': {'id': 2, 'position': 0.8171439440980768, 'ancestral_state': '0', 'mutations': [{'id': 2, 'site': 2, 'node': 1, 'derived_state': '1', 'parent': -1, 'metadata': b''}], 'metadata': b''}, 'alleles': ('0', '1'), 'genotypes': array([0, 1, 0, 0, 0, 0], dtype=uint8), 'position': 0.8171439440980768, 'index': 2}
{'site': {'id': 3, 'position': 2.2212454839609563, 'ancest

A similsr thing: the ```sites()``` iterator.

In [59]:
for site in ts.sites():
    print(site)

{'id': 0, 'position': 0.18288280116394162, 'ancestral_state': '0', 'mutations': [{'id': 0, 'site': 0, 'node': 2, 'derived_state': '1', 'parent': -1, 'metadata': b''}], 'metadata': b''}
{'id': 1, 'position': 0.592431970871985, 'ancestral_state': '0', 'mutations': [{'id': 1, 'site': 1, 'node': 8, 'derived_state': '1', 'parent': -1, 'metadata': b''}], 'metadata': b''}
{'id': 2, 'position': 0.8171439440980768, 'ancestral_state': '0', 'mutations': [{'id': 2, 'site': 2, 'node': 1, 'derived_state': '1', 'parent': -1, 'metadata': b''}], 'metadata': b''}
{'id': 3, 'position': 2.2212454839609563, 'ancestral_state': '0', 'mutations': [{'id': 3, 'site': 3, 'node': 10, 'derived_state': '1', 'parent': -1, 'metadata': b''}], 'metadata': b''}
{'id': 4, 'position': 3.1551563390530646, 'ancestral_state': '0', 'mutations': [{'id': 4, 'site': 4, 'node': 2, 'derived_state': '1', 'parent': -1, 'metadata': b''}], 'metadata': b''}
{'id': 5, 'position': 4.461345006711781, 'ancestral_state': '0', 'mutations': [

Info overload! Here are just the genotypes, printed sequentially:

In [20]:
for var in ts.variants():
    print(var.genotypes)

[0 0 1 0 0 0]
[1 1 0 0 0 0]
[0 1 0 0 0 0]
[1 1 0 1 1 1]
[0 0 1 0 0 0]
[1 1 0 1 1 1]
[0 1 0 0 0 0]
[0 0 0 1 1 1]
[0 0 1 0 0 0]
[0 0 1 0 0 0]
[0 1 0 0 0 0]
[1 0 0 0 0 0]
[0 0 1 0 0 0]
[0 0 1 0 0 0]


Note that `variants()` accesses each *site* sequentially. If you instead wish to see each simulated *sequence* sequentially, use `haplotypes()`:

In [21]:
for hap in ts.haplotypes():
    print(hap)

01010100000100
01110110001000
10001000110011
00010101000000
00010101000000
00010101000000


It is perhaps natural to think of each of these sequences as belonging to a particular individual. By default, msprime simulations do not assume this, so you will see nothing here at the moment. But we will revisit this later!

In [22]:
for ind in ts.individuals():
    print(ind)

In [23]:
ts.num_individuals

0

It may be useful to consider ways to summarise the diversity of this simulated sample. One way to do this is via *pairwise diversity*, the average number of mutations per genome that differ between a randomly chosen pair of samples.

In [32]:
ts.pairwise_diversity()

5.133333333333334

You might also want to calculate this measure for particular subsamples, such as those from particular populations. To do this, simplify specify a list of the desired samples as an argument.

We can see that there is much more diversity within the samples from population 0 compared with our samples from population 1. This is confirmed by the pairwise diversity calculation:

In [33]:
ts.pairwise_diversity(samples=[0,1,2])

8.666666666666666

In [34]:
ts.pairwise_diversity(samples=[3,4,5])

0.0

If the simulated history includes divergences, you will have distinct populations represented by some of the samples. (In fact, we specified this directly with the ```population_configurations``` input to ```msprime.simulate```). Each population is stored in an iterator, ```populations()```:

In [24]:
for pop in ts.populations():
    print(pop)

{'id': 0, 'metadata': b''}
{'id': 1, 'metadata': b''}


You can also access the data corresponding to each population individually (though this isn't very useful at present).

In [25]:
ts.population(1)

{'id': 1, 'metadata': b''}

To see which samples (and nodes) belong to a given population, specify the population id as an argument into ```samples()```:

In [30]:
ts.samples(population=1)

array([3, 4, 5], dtype=int32)

Some kind of inscrutable, but perhaps useful information about the software itself, is found in `provenances()`. It would be useful to see what happens once SLIM is also used to generate some data... perhaps this adds to the provenance tables? Investigate later.

In [27]:
for prov in ts.provenances():
    print(prov)

{'id': 0, 'timestamp': '2018-10-03T09:30:07.151037', 'record': '{"schema_version": "1.0.0", "software": {"name": "msprime", "version": "0.6.1"}, "parameters": {"command": "simulate", "random_seed": 1, "TODO": "add other simulation parameters"}, "environment": {"libraries": {"gsl": {"version": "2.2"}, "kastore": {"version": "0.1.0"}}, "os": {"system": "Darwin", "node": "6200D-132482-M.local", "release": "17.7.0", "version": "Darwin Kernel Version 17.7.0: Thu Jun 21 22:53:14 PDT 2018; root:xnu-4570.71.2~1/RELEASE_X86_64", "machine": "x86_64"}, "python": {"implementation": "CPython", "version": "3.6.6"}}}'}


# Simplifying sequences

Say we wanted to see the history of a subsample of our sample. The ```simplify()``` command allows us to do this, taking the samples of interest as an argument.

In [35]:
ts0 = ts.simplify(samples=[1,3,5])

In [37]:
for tree in ts0.trees():
    print(tree.draw(format="unicode"))

  5    
┏━┻┓   
┃  4   
┃ ┏┻┓  
0 1 2  

  5    
┏━┻┓   
┃  3   
┃ ┏┻┓  
1 0 2  



Note that the sample nodes have been relabelled: 1-->0, 3-->1, 5-->2. The internal nodes have also changed: 9-->4, 7-->3, 10-->5.

To see this mapping, we can specify the argument ```map_nodes=True```. This returns a tuple. The first element of the tuple is the simplified TreeSequence; the second is an array showing this mapping from old nodes to new. Any of the original nodes that are not in the new sequence are represented by -1. 

In [45]:
ts1 = ts.simplify(samples=[1,3,5], map_nodes=True)

print(ts1[1])

[-1  0 -1  1 -1  2 -1  3 -1  4  5 -1]


Note that in removing some samples from the original tree, some sites that were originally segregating may now become non-segregating:

In [60]:
for site in ts0.sites():
    print(site)

{'id': 0, 'position': 0.592431970871985, 'ancestral_state': '0', 'mutations': [{'id': 0, 'site': 0, 'node': 0, 'derived_state': '1', 'parent': -1, 'metadata': b''}], 'metadata': b''}
{'id': 1, 'position': 0.8171439440980768, 'ancestral_state': '0', 'mutations': [{'id': 1, 'site': 1, 'node': 0, 'derived_state': '1', 'parent': -1, 'metadata': b''}], 'metadata': b''}
{'id': 2, 'position': 2.2212454839609563, 'ancestral_state': '0', 'mutations': [{'id': 2, 'site': 2, 'node': 5, 'derived_state': '1', 'parent': -1, 'metadata': b''}], 'metadata': b''}
{'id': 3, 'position': 4.461345006711781, 'ancestral_state': '0', 'mutations': [{'id': 3, 'site': 3, 'node': 5, 'derived_state': '1', 'parent': -1, 'metadata': b''}], 'metadata': b''}
{'id': 4, 'position': 4.562516381453547, 'ancestral_state': '0', 'mutations': [{'id': 4, 'site': 4, 'node': 0, 'derived_state': '1', 'parent': -1, 'metadata': b''}], 'metadata': b''}
{'id': 5, 'position': 5.930655167903751, 'ancestral_state': '0', 'mutations': [{'id

If this is not desired, you can use the ```reduce_to_site_topology=True``` option. 

In [58]:
ts2 = ts.simplify(samples=[1,2,3], reduce_to_site_topology=True)

for tree in ts2.trees():
    print(tree.draw(format="unicode"))

for var in ts2.variants():
    print(var.genotypes)

  4    
┏━┻┓   
┃  3   
┃ ┏┻┓  
1 0 2  

[0 1 0]
[1 0 0]
[1 0 0]
[1 0 1]
[0 1 0]
[1 0 1]
[1 0 0]
[0 0 1]
[0 1 0]
[0 1 0]
[1 0 0]
[0 1 0]
[0 1 0]


In [ ]:
for site in ts